In [1]:
%matplotlib inline

import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')

def pprint(o):
    # small helper to make dict dumps a bit prettier
    print(json.dumps(o, indent=2))

### data.json ahs been already created as well as the config.yml file

## Training the NLU Model

In [2]:
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config


In [3]:
# from rasa_nlu.training_data import load_data
# from rasa_nlu.config import RasaNLUModelConfig
# from rasa_nlu.model import Trainer
# from rasa_nlu import config

# loading the nlu training samples
training_data = load_data("../data/data.json")

# trainer to educate our pipeline
trainer = Trainer(config.load("../config.yml"))

# train the model!
interpreter = trainer.train(training_data)

# store it for future use
model_directory = trainer.persist("./models/nlu", fixed_model_name="current")

INFO:rasa_nlu.training_data.loading:Training data format of ../data/data.json is rasa_nlu
INFO:rasa_nlu.training_data.training_data:Training data stats: 
	- intent examples: 887 (3 distinct intents)
	- Found intents: 'sales', 'goodbye', 'greet'
	- entity examples: 878 (3 distinct entities)
	- found entities: 'sales_entity', 'store_id', 'date'

INFO:rasa_nlu.model:Starting to train component tokenizer_whitespace
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component intent_entity_featurizer_regex
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component ner_crf
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component ner_synonyms
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component intent_featurizer_count_vectors
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component intent_classifie

In [4]:
pprint((interpreter.parse("calculate my cummulative sales in 2014")))

{
  "intent": {
    "name": "sales",
    "confidence": 0.9850747585296631
  },
  "entities": [
    {
      "start": 10,
      "end": 12,
      "value": "my",
      "entity": "store_id",
      "confidence": 0.9979437656331461,
      "extractor": "ner_crf"
    },
    {
      "start": 13,
      "end": 30,
      "value": "cummulative sales",
      "entity": "sales_entity",
      "confidence": 0.8946741200976823,
      "extractor": "ner_crf"
    },
    {
      "start": 34,
      "end": 38,
      "value": "2014",
      "entity": "date",
      "confidence": 0.9981110891988527,
      "extractor": "ner_crf"
    },
    {
      "start": 31,
      "end": 38,
      "text": "in 2014",
      "value": "2014-01-01T00:00:00.000-08:00",
      "confidence": 1.0,
      "additional_info": {
        "values": [
          {
            "value": "2014-01-01T00:00:00.000-08:00",
            "grain": "year",
            "type": "value"
          }
        ],
        "value": "2014-01-01T00:00:00.000-08:00",
    

# Custom Action

In [124]:
from __future__ import absolute_import
from __future__ import division
from __future__ import unicode_literals

from rasa_core.actions.action import Action
from rasa_core.events import SlotSet


class ActionSales(Action):
    def name(self):
        return 'action_sales'

    
    def run(self, dispatcher, tracker, domain):
        store_id = tracker.get_slot('store_id')
        date = tracker.get_slot('time')
        #d1 = date[:10]
        
        time_entity = next((e for e in tracker.latest_message.entities if
                                 e['entity'] == 'time'), None)

        
        ####### Extracting date extracted from the duckling for SQL query
        
        import pyodbc
        import pandas as pd
        import datetime
        import pytz
        import dateutil.parser 
        
        #Check for **duckling_time_dictionary['additional_info']['type']**
        #It would be either 
        #interval
        #value
        
        if time_entity['additional_info']['type'] == 'value':
            
            # Checking grain value for day, week, month, quarter, year
            
            # Day
            
            if time_entity['additional_info']['grain'] == 'day':
                format = '%Y-%m-%dT%H:%M:%S%z'
                datestring = time_entity['additional_info']['value'] 
                d = dateutil.parser.parse(datestring) 
                d1 = d.date() 
                d2 = d.date() + relativedelta(days = +0) # need to change the days=+1
            
            # Week
            elif time_entity['additional_info']['grain'] == 'week':
                format = '%Y-%m-%dT%H:%M:%S%z'
                datestring = time_entity['additional_info']['value']
                d = dateutil.parser.parse(datestring) 
                d1 = d.date() 
                d2 = d.date() + relativedelta(weeks = +1)
                
            # Month
            elif time_entity['additional_info']['grain'] == 'month':
                format = '%Y-%m-%dT%H:%M:%S%z'
                datestring = time_entity['additional_info']['value']
                d = dateutil.parser.parse(datestring) 
                d1 = d.date() 
                d2 = d.date() + relativedelta(months = +1)
                
            # Quarter
            elif time_entity['additional_info']['grain'] == 'quarter':
                format = '%Y-%m-%dT%H:%M:%S%z'
                datestring = time_entity['additional_info']['value']
                d = dateutil.parser.parse(datestring) 
                d1 = d.date() 
                d2 = d.date() + relativedelta(months = +3)
                
            # Year
            elif time_entity['additional_info']['grain'] == 'year':
                format = '%Y-%m-%dT%H:%M:%S%z'
                datestring = time_entity['additional_info']['value']
                d = dateutil.parser.parse(datestring) 
                d1 = d.date() 
                d2 = d.date() + relativedelta(years = +1)
                
            else:
                d1 = None
                d2 = None
                
        elif time_entity['additional_info']['type'] == 'interval':
            format = '%Y-%m-%dT%H:%M:%S%z'
            datestring_from = time_entity['value']['from'] 
            d_from = dateutil.parser.parse(datestring_from) 
            d1 = d_from.date()
            
            datestring_to = time_entity['value']['to']
            d_to = dateutil.parser.parse(datestring_to)
            d2 = d_to.date()
            
        else:
            d1 = None
            d2 = None
            
        ### Creating coonection to database on the basis of date       
            
        if d1 is not None:
            
        
        # Credentilas for connecting to the database
        
        
        #query = "select sum(sales) from DH_ALL_STORES_ALL_DAYS_SALES_PRED_A where store_id = 369 and date = {}".format("'"+ d1 +"'")
        #df = pd.read_sql_query(query,cnxn)
        #response = """It is currently the default message, since the database connection is pending"""
        #response = pd.read_sql_query(query,cnxn)
            response = "Calculated the date and need to connect to database.{},{}".format(d1,d2)
        
        else:
            response = "Kindly provide a valid date for the query"
        
        dispatcher.utter_message("Time entity is  {}".format(response))
		#return [SlotSet('store_id',store_id)]

In [125]:

from  rasa_core.policies import FallbackPolicy, KerasPolicy, MemoizationPolicy
from rasa_core.agent import Agent

# this will catch predictions the model isn't very certain about
# there is a threshold for the NLU predictions as well as the action predictions
# fallback = FallbackPolicy(fallback_action_name="utter_unclear",
#                           core_threshold=0.2,
#                           nlu_threshold=0.1)

agent = Agent('../retail_domain.yml', policies=[MemoizationPolicy(), KerasPolicy()])#, fallback])

# loading our neatly defined training dialogues
training_data = agent.load_data('../data/stories.md')

agent.train(
    training_data,
    validation_split=0.0,
    epochs=200
)

agent.persist('models/dialogue')

/home/pds_gcloud/anaconda3/envs/rasa/lib/python3.6/site-packages/pykwalify/core.py:99: UnsafeLoaderWarning: 
The default 'Loader' for 'load(stream)' without further arguments can be unsafe.
Use 'load(stream, Loader=ruamel.yaml.Loader)' explicitly if that is OK.
Alternatively include the following in your code:

  import warnings
  warnings.simplefilter('ignore', ruamel.yaml.error.UnsafeLoaderWarning)

In most other cases you should consider using 'safe_load(stream)'
  data = yaml.load(stream)
Processed Story Blocks:   0%|          | 0/6 [00:00<?, ?it/s, # trackers=1]ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
Processed Story Blocks: 100%|██████████| 6/6 [00:00<00:00, 384.00it/s, # trackers=1]
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to 

ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
Processed Story Blocks:   0%|          | 0/6 [00:00<?, ?it/s, # trackers=19]ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added 

ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set no

ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
Processed Story Blocks: 100%|██████████| 6/6 [00:00<00:00, 45.13it/s, # trackers=19]
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure y

ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set no

...

329/329 [==============================] - 0s 173us/step - loss: 0.3288 - acc: 0.7964
Epoch 158/200
329/329 [==============================] - 0s 171us/step - loss: 0.3313 - acc: 0.8024
Epoch 159/200
329/329 [==============================] - 0s 172us/step - loss: 0.3380 - acc: 0.7933
Epoch 160/200
329/329 [==============================] - 0s 172us/step - loss: 0.3271 - acc: 0.8237
Epoch 161/200
329/329 [==============================] - 0s 163us/step - loss: 0.3337 - acc: 0.8146
Epoch 162/200
329/329 [==============================] - 0s 175us/step - loss: 0.3427 - acc: 0.7994
Epoch 163/200
329/329 [==============================] - 0s 171us/step - loss: 0.3293 - acc: 0.8085
Epoch 164/200
329/329 [==============================] - 0s 174us/step - loss: 0.3265 - acc: 0.8298
Epoch 165/200
329/329 [==============================] - 0s 182us/step - loss: 0.3358 - acc: 0.8024
Epoch 166/200
329/329 [==============================] - 0s 179us/step - loss: 0.3247 - acc: 0.8024
Epoch 167/200


INFO:rasa_core.policies.keras_policy:Done fitting keras policy model
INFO:rasa_core.agent:Model directory models/dialogue exists and contains old model files. All files will be overwritten.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all yo

ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set no

ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set no

ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set no

In [126]:
from rasa_core.agent import Agent
agent = Agent.load('./models/dialogue', interpreter=model_directory)

/home/pds_gcloud/anaconda3/envs/rasa/lib/python3.6/site-packages/pykwalify/core.py:99: UnsafeLoaderWarning: 
The default 'Loader' for 'load(stream)' without further arguments can be unsafe.
Use 'load(stream, Loader=ruamel.yaml.Loader)' explicitly if that is OK.
Alternatively include the following in your code:

  import warnings
  warnings.simplefilter('ignore', ruamel.yaml.error.UnsafeLoaderWarning)

In most other cases you should consider using 'safe_load(stream)'
  data = yaml.load(stream)
INFO:tensorflow:Restoring parameters from /home/pds_gcloud/chatbot/retail369/Notebook/./models/nlu/default/current/intent_classifier_tensorflow_embedding.ckpt


In [127]:
print("Your bot is ready to talk! Type your messages here or send 'stop'")
while True:
    a = input()
    if a == 'stop':
        break
    responses = agent.handle_message(a)
    for response in responses:
        print(response["text"])

Your bot is ready to talk! Type your messages here or send 'stop'
what is sales between Monday to Saturday of this week in store123
Time entity is  Calculated the date and need to connect to database.2018-08-20,2018-08-26
what is sales between January to march of 2017 in store123
Time entity is  Calculated the date and need to connect to database.2017-01-01,2017-04-01


KeyboardInterrupt: 

# Logic for date - custom action

- import datetime
- import pytz
- import dateutil.parser 



- Check for **duckling_time_dictionary['additional_info']['type']**
- It would be either 
    - **interval**
    - **value**
- If it is **dict['additional_info']['type']** == **value** then check the **grain**

    if **dict['additional_info']['grain'] == 'day'** then
        Date:
        format = '%Y-%m-%dT%H:%M:%S%z'
        datestring = dict['additional_info']['value'] ---> '2018-07-01T16:43:45-07:00'
        d = dateutil.parser.parse(datestring) 
        d1 = d.date() 
        d2 = d.date() + relativedelta(days = 0)
     
     elseif **dict['additional_info']['grain'] == 'week'** then
        format = '%Y-%m-%dT%H:%M:%S%z'
        datestring = dict['additional_info']['value'] ---> '2018-07-01T16:43:45-07:00'
        d = dateutil.parser.parse(datestring) 
        d1 = d.date() 
        d2 = d.date() + relativedelta(weeks = 1)
        
     elseif **dict['additional_info']['grain'] == 'month'** then
        format = '%Y-%m-%dT%H:%M:%S%z'
        datestring = dict['additional_info']['value'] ---> '2018-07-01T16:43:45-07:00'
        d = dateutil.parser.parse(datestring) 
        d1 = d.date() 
        d2 = d.date() + relativedelta(months = 1)
        
     elseif **dict['additional_info']['grain'] == 'quarter'** then
        format = '%Y-%m-%dT%H:%M:%S%z'
        datestring = dict['additional_info']['value'] ---> '2018-07-01T16:43:45-07:00'
        d = dateutil.parser.parse(datestring) 
        d1 = d.date() 
        d2 = d.date() + relativedelta(months = 3)
        
     elseif **dict['additional_info']['grain'] == 'year'** then
        format = '%Y-%m-%dT%H:%M:%S%z'
        datestring = dict['additional_info']['value'] ---> '2018-07-01T16:43:45-07:00'
        d = dateutil.parser.parse(datestring) 
        d1 = d.date() 
        d2 = d.date() + relativedelta(years = 1)
        
  else **dict['additional_info']['type']** == **interval** then
     
     format = '%Y-%m-%dT%H:%M:%S%z'
     datestring_from = dict['value']['from'] ---> '2018-07-01T16:43:45-07:00'
     d_from = dateutil.parser.parse(datestring_from) 
     d1 = d_from.date()
     
     datestring_to = dict['value']['to']
     d_to = dateutil.parser.parse(datestring_to)
     d2 = d_to.date()
     
     
      
        
     
        
        
        


In [116]:
time_entity = {'start': 14, 'end': 24, 'text': 'this month', 'value': '2018-08-01T00:00:00.000-07:00', 'confidence': 1.0, 'additional_info': {'values': [{'value': '2018-08-01T00:00:00.000-07:00', 'grain': 'month', 'type': 'value'}], 'value': '2018-08-01T00:00:00.000-07:00', 'grain': 'month', 'type': 'value'}, 'entity': 'time', 'extractor': 'ner_duckling_http'}

if time_entity is not None:
    print('yes')

yes


In [9]:
# DAY 

day_dict = {  
   'start':24,
   'end':32,
   'text':'tomorrow',
   'value':'2018-08-24T00:00:00.000-07:00',
   'confidence':1.0,
   'additional_info':{  
      'values':[  
         {  
            'value':'2018-08-24T00:00:00.000-07:00',
            'grain':'day',
            'type':'value'
         }
      ],
      'value':'2018-08-24T00:00:00.000-07:00',
      'grain':'day',
      'type':'value'
   },
   'entity':'time',
   'extractor':'ner_duckling_http'
}

In [10]:
day_dict.keys()

dict_keys(['start', 'end', 'text', 'value', 'confidence', 'additional_info', 'entity', 'extractor'])

In [11]:
day_dict['additional_info'].keys()

dict_keys(['values', 'value', 'grain', 'type'])

In [17]:
day_dict['additional_info']['type']

'value'

In [100]:
# Check the grain value 
day_dict['additional_info']['value']

'2018-08-24T00:00:00.000-07:00'

In [13]:
# Interval Date
interval_dict = {  
   'start':13,
   'end':44,
   'text':'from january to march this year',
   'value':{  
      'to':'2018-04-01T00:00:00.000-07:00',
      'from':'2018-01-01T00:00:00.000-08:00'
   },
   'confidence':1.0,
   'additional_info':{  
      'values':[  
         {  
            'to':{  
               'value':'2018-04-01T00:00:00.000-07:00',
               'grain':'month'
            },
            'from':{  
               'value':'2018-01-01T00:00:00.000-08:00',
               'grain':'month'
            },
            'type':'interval'
         },
         {  
            'to':{  
               'value':'2018-04-01T00:00:00.000-07:00',
               'grain':'month'
            },
            'from':{  
               'value':'2017-01-01T00:00:00.000-08:00',
               'grain':'month'
            },
            'type':'interval'
         },
         {  
            'to':{  
               'value':'2018-04-01T00:00:00.000-07:00',
               'grain':'month'
            },
            'from':{  
               'value':'2016-01-01T00:00:00.000-08:00',
               'grain':'month'
            },
            'type':'interval'
         }
      ],
      'to':{  
         'value':'2018-04-01T00:00:00.000-07:00',
         'grain':'month'
      },
      'from':{  
         'value':'2018-01-01T00:00:00.000-08:00',
         'grain':'month'
      },
      'type':'interval'
   },
   'entity':'time',
   'extractor':'ner_duckling_http'
}

In [14]:
interval_dict.keys()

dict_keys(['start', 'end', 'text', 'value', 'confidence', 'additional_info', 'entity', 'extractor'])

In [15]:
interval_dict['additional_info'].keys()

dict_keys(['values', 'to', 'from', 'type'])

In [114]:
interval_dict['value']['to']

'2018-04-01T00:00:00.000-07:00'

In [16]:
interval_dict['additional_info']['type']

'interval'

In [21]:
# Playing for quarter code
# using dateutil
from dateutil.relativedelta import relativedelta
import datetime

def getQuarterStart(dt=datetime.date.today()):
    return datetime.date(dt.year, (dt.month - 1) // 3 * 3 + 1, 1)

def getQuarterEnd2(dt=datetime.date.today()):
    quarterStart = getQuarterStart(dt)
    return quarterStart + relativedelta(months=3, days=-1)

In [27]:
getQuarterStart(datetime.date.today()) + relativedelta(months=3, days=-1)

datetime.date(2018, 9, 30)

In [30]:
?relativedelta(months=3, days=-1)

In [22]:
getQuarterEnd2()

datetime.date(2018, 9, 30)

In [48]:
import pytz
import dateutil.parser

In [49]:
dateutil.parser.parse('2018-01-01T00:00:00.000-08:00')

datetime.datetime(2018, 1, 1, 0, 0, tzinfo=tzoffset(None, -28800))

In [57]:
import pytz

In [69]:
datetime.date.today()

datetime.date(2018, 8, 24)

In [88]:
d.date()

datetime.date(2018, 7, 1)

In [110]:
d.date() + relativedelta(years=1)

datetime.date(2019, 7, 1)

In [102]:
format = '%Y-%m-%dT%H:%M:%S%z'
datestring = '2018-07-01T16:43:45-07:00'
d = dateutil.parser.parse(datestring) # python 2.7
#d = d.replace(tzinfo=utc) - d.utcoffset()
print(d.date() + relativedelta(months = 3))

2018-10-01
